In [ ]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    langgraph \
    httpx

In [ ]:
#PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#LOCATION = "us-central1"  # @param {type:"string"}
#STAGING_BUCKET = "gs://[your-staging-bucket]"  # @param {type:"string"}
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [ ]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
from vertexai import agent_engines

In [ ]:
@tool
def get_product_details(product_name: str):
    """Gathers basic details about a product"""
    details = {
        "smartphone": "A cutting edge smartphone with advanced camera features and lightning-fast processing.",
        "coffee": "A rich, Aromatic blend of ethically sourced coffee beans.",
        "Shoes": "High-performance running shoes designed for comfort, support, and speed.",        
    }

    return details.get(product_name, "Product details not found.")

In [ ]:
def router(state: list[BaseMessage]) -> Literal["get_product_details","__end__"]:
    """Initiates the product details retreival if the user asks for a product."""
    #Get the tool_calls from the last message in the conversation history.
    tool_calls = state[-1].tool_calls
    if len(tool_calls):
        return "get_product_details"
    else:
        #End the conversation flow.
        return "__end__"

In [ ]:
class SimpleLangGraphApp:
    def __init__(self, project: str, location: str) -> None:
        self.project_id = project 
        self.location = location

# The set_up method is used to define application initialization logic
    def set_up(self) -> None:
        model = ChatVertexAI(model="gemini-2.0-flash")

        builder = MessageGraph()

        model_with_tools = model.bind_tools([get_product_details])
        builder.add_node("tools", model_with_tools)

        tool_node = ToolNode([get_product_details])
        builder.add_node("get_product_details", tool_node)
        builder.add_edge("get_product_details", END)

        builder.set_entry_point("tools")
        builder.add_conditional_edges("tools", router)
        self.runnable = builder.compile()

    def query(self, message:str):
        """Query the application.
        
        Args: 
            message: The user message.
        
        Returns:
            str: The LLM response.
        """

        chat_history = self.runnable.invoke(HumanMessage(message))

        return chat_history[-1].content

In [ ]:
agent = SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION)
agent.set_up()

In [ ]:
agent.query(message="Get product details for smartphone")

In [ ]:

agent.query(message="Get product details for coffee")

In [ ]:

agent.query(message="Get product details for Shoes")

In [ ]:
remote_agent = agent_engines.create(
    SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION),
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "langgraph",
        "httpx",
    ],
    display_name="Agent Engine with LangGraph",
    description="This is a sample custom application in Agent Engine that uses LangGraph",
    extra_packages=[],
)

In [ ]:
remote_agent.query(message="Get product details for coffee")

In [ ]:
remote_agent.delete()